In [1]:
"""
代码原作者：Sriram Ravindran, sriram@ucsd.edu
https://github.com/aliasvishnu/EEGNet
Original paper - https://arxiv.org/abs/1611.08024
修改部分细节并注释：向羽 11810203@mail.sustc.edu.cn
原代码使用的torch 版本比较老旧，本人EEG识别新手，拿这篇代码练练手
开始时间：2020.7.26
有问题可以联系我，虽然我也不一定会哈
"""


import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim as optim
###导入必要的库###

In [3]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet,self).__init__()
        self.T = 120   ###设置采样点数###
        
        #layer 1 一维卷积
        self.conv1 = nn.Conv2d(1,16,(1,64),padding=0)
        ###第一层输入1通道，输出16通道，卷积核（1，64），不补零###
        self.batchnorm1 = nn.BatchNorm2d(16,False)
        ###数据归一化，便于处理###
        
        
        #layer 2  2维卷积
        self.padding1 = nn.ZeroPad2d((16,17,0,1))
        ###补零，左右上下###
        self.conv2 = nn.Conv2d(1,4,(2,32))
        self.batchnorm2 = nn.BatchNorm2d(4)
        self.pooling2 = nn.MaxPool2d(2,4)
        ###最大池化###
        
        #layer 3
        self.padding2 = nn.ZeroPad2d((2,1,4,3))
        self.conv3 = nn.Conv2d(4,4,(8,4))
        self.batchnorm3 = nn.BatchNorm2d(4)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        #FC layer
        self.fc1 = nn.Linear(4*2*7,1)
    
    def forward(self,x):
        #layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x,0.25)
        x = x.permute(0,3,1,2)
        #permute 换位？有点迷 输出应该是三维来着，这个换位换了四维
        
        #layer 2 
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x,0.25)
        x = self.pooling2(x)
        
        #layer 3 
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm2(x)
        x = F.dropout(x,0.25)
        x = self.pooling3(x)
        
        #fc layer
        x = x.view(-1,4*2*7) #改变x的维度
        x = F.sigmoid(self.fc1(x))
        return x
   
net = EEGNet()
print (net.forward(torch.Tensor(np.random.rand(1, 1, 120, 64))))
#随便找个数据测试下
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters())    
"""
本次实验环境为GPU环境，如果需要GPU加速可以在网络和数据上+.cuda()
"""

c:\users\13900\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[0.6822]], grad_fn=<SigmoidBackward>)


'\n本次实验环境为GPU环境，如果需要GPU加速可以在网络和数据上+.cuda()\n'

In [11]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 100
    
    predicted = []
    
    for i in range(len(X)//batch_size):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X[s:e])
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
    inputs = torch.from_numpy(X)
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results

In [5]:

X_train = np.random.rand(100, 1, 120, 64).astype('float32') # np.random.rand generates between [0, 1)
y_train = np.round(np.random.rand(100).astype('float32')) # binary data, so we round it to 0 or 1.

X_val = np.random.rand(100, 1, 120, 64).astype('float32')
y_val = np.round(np.random.rand(100).astype('float32'))

X_test = np.random.rand(100, 1, 120, 64).astype('float32')
y_test = np.round(np.random.rand(100).astype('float32'))

In [12]:
batch_size = 32
# 训练 循环
for epoch in range(10): 
    print("\nEpoch ", epoch)

    running_loss = 0.0
    for i in range(len(X_train)//batch_size-1):
        s = i*batch_size
        e = i*batch_size+batch_size

        inputs = torch.from_numpy(X_train[s:e])
        labels = torch.FloatTensor(np.array([y_train[s:e]]).T*1.0)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()

    # 验证
    params = ["acc", "auc", "fmeasure"]
    print(params)
    print("Training Loss ", running_loss)
    print("Train - ", evaluate(net, X_train, y_train, params))
    print("Validation - ", evaluate(net, X_val, y_val, params))
    print("Test - ", evaluate(net, X_test, y_test, params))


Epoch  0
['acc', 'auc', 'fmeasure']
Training Loss  1.6255861520767212
Train -  [0.43, 0.427536231884058, 0.5210084033613445]


c:\users\13900\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Validation -  [0.52, 0.49855312112443156, 0.6307692307692307]


c:\users\13900\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Test -  [0.44, 0.4257703081232493, 0.5483870967741935]

Epoch  1


...

['acc', 'auc', 'fmeasure']
Training Loss  1.3194060921669006
Train -  [0.6, 0.607085346215781, 0.5555555555555556]


c:\users\13900\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Validation -  [0.5, 0.5233567589913187, 0.5370370370370371]


c:\users\13900\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Test -  [0.4, 0.4125650260104041, 0.4117647058823529]
